library

In [5]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization

load data

In [9]:
data = pd.read_csv('/content/Cardiovascular_Disease_Dataset.csv')
data.head()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,103368,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,119250,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,119372,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,132514,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,146211,31,1,1,199,0,0,2,136,0,5.3,3,2,1


In [10]:
data = data.drop(columns='patientid', axis=1)
data.head()

,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,31,1,1,199,0,0,2,136,0,5.3,3,2,1


## pisahkan data atribut dengan label

In [12]:
x = data.drop(columns = 'target', axis = 1)
y = data['target']

In [13]:
print(x)

     age  gender  chestpain  restingBP  serumcholestrol  fastingbloodsugar  \
0     53       1          2        171                0                  0   
1     40       1          0         94              229                  0   
2     49       1          2        133              142                  0   
3     43       1          0        138              295                  1   
4     31       1          1        199                0                  0   
..   ...     ...        ...        ...              ...                ...   
995   48       1          2        139              349                  0   
996   47       1          3        143              258                  1   
997   69       1          0        156              434                  1   
998   45       1          1        186              417                  0   
999   25       1          0        158              270                  0   

     restingrelectro  maxheartrate  exerciseangia  oldpeak  slo

In [14]:
print(y)

0      1
1      0
2      0
3      1
4      1
      ..
995    1
996    0
997    1
998    1
999    0
Name: target, Length: 1000, dtype: int64


## Split data train dan data testing

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
print(x.shape, x_train.shape, x_test.shape)

(1000, 12) (800, 12) (200, 12)


##mendefinisikan Fungsi Evaluasi

In [20]:
def xgb_evaluate(max_depth, learning_rate, n_estimators, gamma, min_child_weight, subsample, colsample_bytree):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'gamma': gamma,
        'min_child_weight': min_child_weight,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
    model = XGBClassifier(**params)
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy


##menentukan Batas Parameter yang Mau Dicari

In [21]:
# Set the parameter bounds for Bayesian Optimization
param_bounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (100, 500),
    'gamma': (0, 5),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1)
}

menjalankan Bayesian Optimization-nya

In [22]:
# Perform Bayesian Optimization
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=42
)
optimizer.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:05:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


| 1         | 0.945     | 0.6873    | 4.754     | 0.2223    | 7.191     | 2.404     | 162.4     | 0.529     |


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:05:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


| 2         | 0.945     | 0.9331    | 3.006     | 0.2153    | 3.144     | 9.729     | 433.0     | 0.6062    |
| 3         | 0.955     | 0.5909    | 0.917     | 0.09823   | 6.673     | 4.888     | 216.5     | 0.8059    |


...

| 27        | 0.975     | 0.6926    | 1.946     | 0.2079    | 3.363     | 2.135     | 495.7     | 0.9357    |


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:05:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


| 28        | 0.975     | 0.9617    | 0.6822    | 0.1542    | 3.132     | 1.346     | 498.3     | 0.8119    |


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:06:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:06:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


| 29        | 0.97      | 0.5348    | 1.765     | 0.2873    | 5.218     | 3.054     | 499.9     | 0.83      |
| 30        | 0.965     | 0.5585    | 0.1158    | 0.2472    | 3.108     | 2.725     | 498.5     | 0.8038    |


 Membuat dan melatih model XGBoost dengan parameter terbaik hasil dari Bayesian Optimization

In [24]:
# Extract the best parameters and train the final model
best_params = optimizer.max['params']
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])

final_model = XGBClassifier(
    **best_params,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False
)
final_model.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:08:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=np.float64(0.7256842768725286), device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=np.float64(2.956935218149535), grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=np.float64(0.29586032343178603), max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=np.float64(2.0256442495026943), missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=495,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [32]:
# Evaluate the final model
final_predictions = final_model.predict(x_test)
final_accuracy = accuracy_score(y_test, final_predictions)
print(f"Final Model Accuracy: {final_accuracy}")

Final Model Accuracy: 0.98


In [33]:
# Prediction for a single input
input_data = (53, 1, 2, 171, 0, 0, 1, 147, 0, 5.3, 3, 3)
input_data_as_numpy_array = np.asarray(input_data)
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Use 'final_model' instead of 'model'
prediction = final_model.predict(input_data_reshaped)
print(prediction)

if prediction[0] == 0:
    print('Pasien Tidak Terkena Penyakit Jantung')
else:
    print('Pasien Terkena Penyakit Jantung')

[1]
Pasien Terkena Penyakit Jantung


In [34]:
import pickle

In [36]:
filename = 'penyakit_jantung_koroner.sav'
pickle.dump(final_model, open(filename, 'wb'))